# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
import requests
import json
import matplotlib.pyplot as plt
import pandas as pd
from citipy import citipy
import numpy as np
from config import api_key
from pprint import pprint
import time

## Generate Cities List

In [5]:
# Get random numbers for latitude and longitude then place in a dataframe

city_df=pd.DataFrame({"long":(np.random.uniform(-180,180,70)), "lat":(np.random.uniform(-90,90,70))})
# city_df.head()
# city_df.tail()
print(city_df)

          long        lat
0   162.460435 -38.204444
1   -75.113445 -42.441259
2  -108.152941  36.962380
3   -12.254819 -74.071369
4   156.569095  63.952926
..         ...        ...
65   10.521689  10.940247
66  121.701658  38.938822
67  -17.111081 -80.444665
68   36.278380 -51.029903
69 -176.575816 -89.889222

[70 rows x 2 columns]


# Use Citipy to Retrieve City Names

In [10]:
# Create lists to store city and country names
# Source: https://github.com/wingchen/citipy
citi=[]
country=[]

for index, row in city_df.iterrows():
    latitude=row['lat']
    longitude=row['long']
    city=citipy.nearest_city(latitude,longitude)
    cityname=city.city_name
    cntryname=city.country_code
    if cityname not in citi:
        citi.append(cityname)
        country.append(cntryname)
    else:
        pass
print(citi)
print(country)

# Adds new lists to dataframe
tudes={"City":citi,"Country":country}

# Make a dataframe from the dictionary
tudes_df=pd.DataFrame(tudes)
tudes_df

['te anau', 'castro', 'aztec', 'cape town', 'omsukchan', 'kalmunai', 'lewistown', 'tautira', 'puerto ayora', 'bredasdorp', 'sitka', 'bluff', 'punta arenas', 'busselton', 'isangel', 'ushuaia', 'atuona', 'flinders', 'kusye-aleksandrovskiy', 'port moresby', 'morehead', 'dourbali', 'saskylakh', 'bairiki', 'mataura', 'butaritari', 'ozieri', 'bolshiye uki', 'new norfolk', 'illoqqortoormiut', 'vaini', 'belushya guba', 'hambantota', 'vostok', 'zalantun', 'rolim de moura', 'mahibadhoo', 'lerwick', 'manokwari', 'buqayq', 'kurchum', 'rikitea', 'albany', 'hermanus', 'tasiilaq', 'san cristobal', 'vaitupu', 'indianola', 'hervey bay', 'tocopilla', 'brae', 'pisco', 'barrow', 'kinablangan', 'hobart', 'yellowknife', 'ahipara', 'the pas', 'duku', 'dalianwan', 'port alfred']
['nz', 'cl', 'us', 'za', 'ru', 'lk', 'us', 'pf', 'ec', 'za', 'us', 'nz', 'cl', 'au', 'vu', 'ar', 'pf', 'au', 'ru', 'pg', 'pg', 'td', 'ru', 'ki', 'pf', 'ki', 'it', 'ru', 'au', 'gl', 'to', 'ru', 'lk', 'ru', 'cn', 'br', 'mv', 'gb', 'id',

,City,Country
0,te anau,nz
1,castro,cl
2,aztec,us
3,cape town,za
4,omsukchan,ru
...,...,...
56,ahipara,nz
57,the pas,ca
58,duku,ng
59,dalianwan,cn


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [11]:
# https://api.openweathermap.org/data/2.5/weather?q=London,uk&appid=api_key&units=imperial
# https://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=api_key&q=London,uk

url = "http://api.openweathermap.org/data/2.5/weather?"
units = 'imperial'
query_url=f"{url}&units={units}&appid={api_key}&q="
# query_url = f"{url}appid={api_key}&units={units}&q="

cityid = []
long = []
lat = []
temp = []
humidity = []
clouds = []
wind = []
maxtemp = []
date = []
cityagain=[]
countryagain=[]
# # Build query URL
# check if data is available before appending
# for City in city_df:
#     response=requests.get(query_url + cityposition +","+ cntryposition).json()
    
#     try:
#         if response["message"]=="city not found":
#             print("The city can't be found")
#     except:
#         cityagain.append(City)
#         countryagain.append(response['sys']['country'])
#         cityid.append(response['id'])
#         long.append(response['coord']['lon'])
#         lat.append(response['coord']['lat'])
#         temp.append(response['main']['temp'])
#         humidity.append(response['main']['humidity'])
#         clouds.append(response['clouds']['all'])
#         wind.append(response['wind']['speed'])
#         maxtemp.append(response['main']['temp_max'])
#         date.append(response['dt'])
#         print(f'The city name is: {city}')
#         print(f'The city ID is: {response["id"]}')
#         print(f'The longitude information received is: {response["coord"]["lon"]}')
#         print(f'The latitude information received is: {response["coord"]["lat"]}')
#         print(f'The longitude information received is: {response["coord"]["lon"]}')
#         print(f'The temperature is: {response["main"]["temp"]}')
#         print(f'The humidity is: {response["main"]["humidity"]}')
#         print(f'The cloudiness is: {response["clouds"]["all"]}')
#         print(f'The wind speed is: {response["wind"]["speed"]}')
#         print(f'The high is: {response["main"]["temp_max"]}')
#         print(f"                          ")
        
for index, row in tudes_df.iterrows():
    cityposition=row['City']
    cntryposition= row['Country']
    response=requests.get(query_url + cityposition +","+ cntryposition).json()
    # time.sleep(1)
    # pprint(response)
    
    try:
        if (response["message"]=="city not found"):
            print(f"The city can't be found")
            print(f"                          ")
            # pass
                  
    except:
        if len(cityid)<50:
            cityagain.append(response['name'])
            countryagain.append(response['sys']['country'])
            cityid.append(response['id'])
            long.append(response['coord']['lon'])
            lat.append(response['coord']['lat'])
            temp.append(response['main']['temp'])
            humidity.append(response['main']['humidity'])
            clouds.append(response['clouds']['all'])
            wind.append(response['wind']['speed'])
            maxtemp.append(response['main']['temp_max'])
            date.append(response['dt'])
            print(f'The city name is: {response["name"]}')
            print(f'The country is: {response["sys"]["country"]}')
            print(f'The city ID is: {response["id"]}')
            print(f'The longitude information received is: {response["coord"]["lon"]}')
            print(f'The latitude information received is: {response["coord"]["lat"]}')
            print(f'The longitude information received is: {response["coord"]["lon"]}')
            print(f'The temperature is: {response["main"]["temp"]}')
            print(f'The humidity is: {response["main"]["humidity"]}')
            print(f'The cloudiness is: {response["clouds"]["all"]}')
            print(f'The wind speed is: {response["wind"]["speed"]}')
            print(f'The high is: {response["main"]["temp_max"]}')
            print(f"                          ")
        else:
            break
# cityposition not in cityagain and len(cityid)<500:

The city name is: Te Anau
The country is: NZ
The city ID is: 2181625
The longitude information received is: 167.72
The latitude information received is: -45.42
The longitude information received is: 167.72
The temperature is: 39.92
The humidity is: 88
The cloudiness is: 35
The wind speed is: 3.38
The high is: 39.92
                          
The city name is: Castro
The country is: CL
The city ID is: 3896218
The longitude information received is: -73.77
The latitude information received is: -42.47
The longitude information received is: -73.77
The temperature is: 50.32
The humidity is: 100
The cloudiness is: 75
The wind speed is: 12.75
The high is: 55.99
                          
The city name is: Aztec
The country is: US
The city ID is: 5456049
The longitude information received is: -107.99
The latitude information received is: 36.82
The longitude information received is: -107.99
The temperature is: 36.55
The humidity is: 38
The cloudiness is: 1
The wind speed is: 9.17
The high is: 39

The city can't be found
                          
The city name is: New Norfolk
The country is: AU
The city ID is: 2155415
The longitude information received is: 147.06
The latitude information received is: -42.78
The longitude information received is: 147.06
The temperature is: 40.69
The humidity is: 73
The cloudiness is: 0
The wind speed is: 0.98
The high is: 41
                          
The city can't be found
                          
The city name is: Vaini
The country is: TO
The city ID is: 4032243
The longitude information received is: -175.2
The latitude information received is: -21.2
The longitude information received is: -175.2
The temperature is: 71.6
The humidity is: 73
The cloudiness is: 40
The wind speed is: 10.29
The high is: 71.6
                          
The city can't be found
                          
The city name is: Hambantota
The country is: LK
The city ID is: 1244926
The longitude information received is: 81.12
The latitude information received is: 6.12
The

The city name is: The Pas
The country is: CA
The city ID is: 6164499
The longitude information received is: -101.25
The latitude information received is: 53.83
The longitude information received is: -101.25
The temperature is: 37.4
The humidity is: 64
The cloudiness is: 90
The wind speed is: 23.04
The high is: 37.4
                          


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [12]:
# Create dictionary of lists above
weather={"City":cityagain,"Country":countryagain,"City ID":cityid,"Latitude":lat,"Longitude":long,"Temperature":temp,"Max Temp":maxtemp,"Humidity":humidity,"Wind":wind,"Clouds":clouds,"Date":date}

# Make a dataframe from the dictionary
weather_df=pd.DataFrame(weather)
weather_df

# Save the updated dataframe as a csv
weather_df.to_csv("../output_data/weather_file.csv", encoding="utf-8", index=False)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
print(cityposition)

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# print(f"The ID of the city is: {cityid}")
# print(f"The latitude information received is: {lat}")
# print(f"The longitude information received is: {long}")
# print(f"The temperature information received is: {temp}")
# print(f"The humidity information received is: {humidity}")
# print(f"The cloud information received is: {clouds}")
# print(f"The wind speed information received is: {wind}")
# print(f"The maximum temperature received is: {maxtemp}")
# print(f"The date received is: {date}")
# print(f"                                                ")
    
    
# url = "http://api.openweathermap.org/data/2.5/weather?"
# units = "imperial"

# # # Build partial query URL
# query_url = f"{url}appid={api_key}&units={units}&q="

#     country=city.country_code
#     country.append(country)
# print(citi)
# print(country)
# cities = ["Paris", "London", "Oslo", "Beijing"]

# url="http://api.openweathermap.org/data/2.5/weather?lat={87.6298}&lon={41.8781}&appid={api_key}"
# # url="http://api.openweathermap.org/data/2.5/weather?q={Chicago}&appid={api_key}?"
# citydata = requests.get(url).json()
# print(json.dumps(citydata, indent=4, sort_keys=True))

# Print the json
# print(geo_data)
# response=requests.get(f"{query_url}'lat='{87.6298}'&lon='{41.8781}").json()
# print(response)

# # set up lists to hold reponse info
# citi = []
# cityid = []
# long = []
# lat = []
# temp = []
# humidity = []
# clouds = []
# wind = []
# country = []

# for index, row in city_df.iterrows():
#     latitude=row['lat']
#     longitude=row['long']
#     response=requests.get(f"{query_url}'lat='{latitude}'&lon='{longitude}").json()
#     print(response)
#     citi.append(response['name'])
#     cityid.append(response['sys']['id'])
#     long.append(response['coord']['lon'])
#     lat.append(response['coord']['lat'])
#     temp.append(response['main']['temp'])
#     humidity.append(response['main']['humidity'])
#     clouds.append(response['clouds']['all'])
#     wind.append(response['wind']['speed'])
#     country.append(response['sys']['country'])

# Loop through the list of cities and perform a request for data on each
# api.openweathermap.org/data/2.5/find?q=London&units=imperial
# for city in cities:
    
#     response = requests.get(query_url + city).json()
#     citi.append(response['name'])
#     cityid.append(response['sys']['id'])
#     long.append(response['coord']['lon'])
#     lat.append(response['coord']['lat'])
#     temp.append(response['main']['temp'])
#     humidity.append(response['main']['humidity'])
#     clouds.append(response['clouds']['all'])
#     wind.append(response['wind']['speed'])
#     country.append(response['sys']['country'])
    
# print(f"The ID of the city is: {cityid}")
# print(f"The city information received is: {citi}")
# print(f"The country information received is: {country}")
# print(f"The latitude information received is: {lat}")
# print(f"The longitude information received is: {long}")
# print(f"The temperature information received is: {temp}")
# print(f"The humidity information received is: {humidity}")
# print(f"The cloud information received is: {clouds}")
# print(f"The wind speed information received is: {wind}")



# # Save config information
# url = "http://api.openweathermap.org/data/2.5/weather?"
# city = "Chicago"

# # Build query URL
# query_url = url + "appid=" + api_key + "&q=" + city

# # Get weather data
# weather_response = requests.get(query_url)
# weather_json = weather_response.json()

# # Get the temperature from the response
# print(f"The weather API responded with: {weather_json}.")
# pprint(weather_json)